# Brightway adapter regionalization demo

In this demo, we go through the configuration for the brightway adapter that demonstrates how to get regionalized results.

First we load the configuration file.

In [1]:
import json

config = json.load(open("config.json", encoding="utf-8"))

First it is important that we specify the brightway project we want to operate on in the adapter configuration

In [2]:
config["adapters"][0]["config"]["bw_project"]
# use
# config["adapters"][0]["config"]["bw_project"] = "<something else>"
# to assign it

'ecoinvent_391'

Rationalizations are defined by hierarchical location names, which are described in lists of strings for each activity.
Setting the regions of brightway activities we have two possibilities. Activities that are included in the 'hierarchy' can include their location in their config with field `enb_location`. Like so:


In [3]:
config["hierarchy"]["children"][0]["config"]

{'name': 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
 'unit': 'kilowatt hour',
 'location': 'DK',
 'enb_location': ['EU', 'DK'],
 'default_output': {'unit': 'kWh', 'magnitude': 1}}

For all other activities, for which the functional units are not required,  we can specify regions lists in the field `set_node_regions`, which is a dictionary, where the keys are brightway activity codes and the values are lists of region names. (Note that it's required but under normal circumstances the case that brightway activities in one project have unique code, tho it's not strickty required for brightway, since it also uses activities database as identifier, which we omit here, under the given assumption). In case, the Brightway adapter contains a helper funciton that can be used to validate that all codes in a brighway project are unique:

 `BrightwayAdapter.assert_all_codes_unique(raise_error: Optional[bool] = False, bw_project: Optional[str] = None)`
Which uses the current bw_project if not specified and writes a warning (or raises an error) if there are duplicate codes.

It is also important to note, that a region that is described as a string, must be on the same index on all specified lists it is included. For example here:

```
{
"activity_0": ["EU", "DK"],
"activity_1": ["EU", "SW"],
"activity_4": ["EU", "ES", "CAT"],
"activity_5": ["EU", "ES", "BAC"],
}
```  

but not

```
{
"activity_0": ["EU", "DK"],
"activity_1": ["ES", "CAT"],
"activity_2": ["EU", "ES", "BAC"],
}
```  

since "ES" (Spain) appears on index 1 for activity 1 but on index 2 for activity 2.


Lets clear this for now: 

In [4]:
config["adapters"][0]["config"]["simple_regionalization"]["set_node_regions"] = {}

Let's look at the rest of the configuration for regionalization:

In [5]:
config["adapters"][0]["config"]["simple_regionalization"]

{'run_regionalization': True,
 'select_regions': ['EU'],
 'set_node_regions': {},
 'clear_all_other_node_regions': True}

These are the parameters:
```
# is the regionalization should be applied (default: false)
run_regionalization: boolean

# the regions of which we want the regionalization impacts of. a list of strings, which must be included in the defined locations. Which levels of the regional are selected does not matter. (e.g. it could be ["EU", "DK", "CAT]) and those regions would be included in the results.
select_regions: list of regions
# As explained above, locations of activities not included in the hierarchy 
set_node_regions: dict of regions (as described above)
# all other activities get their 'enb_location' field deleted, which acts as a reset (default: false).
clear_all_other_node_regions: boolean
```
The locations that brightway stores in activities coming from Ecoinvent in the activities under the key: "location", is not considered for the enbios regionalization. 


Finally, we need to select the regions, of which we want the regionalized results of. This is done by setting the `select_regions` field in the configuration. This field is list of strings, which can be any set of region strings we used for the activity locations. 

In [6]:
config["adapters"][0]["config"]["simple_regionalization"]["select_regions"] = ["DK"]

Create an experiment with the config...

In [7]:
from enbios import Experiment

exp = Experiment(config)
exp

Excluding 0 filtered results
Excluding 104 filtered results


Experiment: (call info() for details)
Structural nodes: 1
Methods: 1
Hierarchy (depth): 2
Scenarios: 1

And run it...

In [8]:
exp.run()

2024-06-04 13:17:58,236 - demos.brightway_adapter_demos.regionalization_demo.enbios.base - INFO - Running scenario 'default scenario'
2024-06-04 13:18:39,544 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'default scenario': {'name': 'root',
  'results': {'GWP1000.DK': {'unit': 'kg CO2-Eq',
    'magnitude': 1.5298065586599818}},
  'output': [{'unit': 'kilowatt_hour', 'magnitude': 1.0, 'label': None}],
  'children': [{'name': 'single_activity',
    'results': {'GWP1000.DK': {'unit': 'kg CO2-Eq',
      'magnitude': 1.5298065586599818}},
    'output': [{'unit': 'kilowatt_hour', 'magnitude': 1.0, 'label': None}],
    'bw_activity_code': 'b9d74efa4fd670b1977a3471ec010737'}]}}

Lets take some activities of the background system and move them to Denmark too and rerun the experiment

In [9]:
from bw2data.backends import Activity

# Get the bw-adapter and its activities; get the first and only activity
activity = list(exp.get_adapter_by_name("brightway-adapter").activityMap.values())[0].bw_activity


def recursively_collect_activities(activity, level: int = 1) -> list[Activity]:
    if level == 0:
        return [activity]
    else:
        res = []
        for exc in activity.exchanges():
            res.extend(recursively_collect_activities(exc.input, level - 1))
        return list(set(res))

move_to_denmark = recursively_collect_activities(activity, 3)
print(f"{len(move_to_denmark)} activities to move to Denmark")
# get its brightway Activity's exchanges
for act in move_to_denmark:
    config["adapters"][0]["config"]["simple_regionalization"]["set_node_regions"][act["code"]] = ["EU", "DK"]

# 
# config
exp = Experiment(config)
exp.run()

905 activities to move to Denmark
Excluding 0 filtered results
Excluding 104 filtered results
2024-06-04 13:18:55,439 - demos.brightway_adapter_demos.regionalization_demo.enbios.base - INFO - Running scenario 'default scenario'
2024-06-04 13:19:44,714 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'default scenario': {'name': 'root',
  'results': {'GWP1000.DK': {'unit': 'kg CO2-Eq',
    'magnitude': 1.5390792603316303}},
  'output': [{'unit': 'kilowatt_hour', 'magnitude': 1.0, 'label': None}],
  'children': [{'name': 'single_activity',
    'results': {'GWP1000.DK': {'unit': 'kg CO2-Eq',
      'magnitude': 1.5390792603316303}},
    'output': [{'unit': 'kilowatt_hour', 'magnitude': 1.0, 'label': None}],
    'bw_activity_code': 'b9d74efa4fd670b1977a3471ec010737'}]}}

In [10]:
from enbios.bw2.brightway_experiment_adapter import BrightwayAdapter
from enbios import Experiment

exp = Experiment(config)
bw_adapter: BrightwayAdapter = exp.get_adapter_by_name("brightway-adapter")
bw_adapter.prepare_regionalization()

Excluding 0 filtered results
Excluding 104 filtered results



When the `run_regionalization` is set to true, enbios will do modifications to the activities as they ar stored in brightway.
All nodes, which are defined in the hierarchy with an `enb_locations` will get that set during the node validation done by the brightway adapter. All other activities and the reset (clear_all_other_node_regions) are performed before the first scenario is run.


## Step by step changes that enbios applies to brightway activities
Enbios will search for all relevant activities and also set the key 'enb_location', which is not use by brightway for anything accordingly. When the LCI is calculated it splits the LCI matrix by regions, before applying the characterization, so that it can give regionalized impacts.  

In [11]:
import bw2data

# first select the project
bw2data.projects.set_current("ecoinvent_391")
bw2data.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent_391_cutoff

In [12]:
# but we also select the right database before searching for activities
db = bw2data.Database("ecoinvent_391_cutoff")

In [13]:
# let's look at the specif of the activity in our hierarchy, as they are needed to pinpoint the activity we are looking for.
# Consider that in other examples, we simply used the activities code to identify activities. However, with a combination of name, location and unit, enbioes is able to filter candidates accordingly and works if the result is one single activity. 
activity_config = config["hierarchy"]["children"][0]["config"]
activity_config

{'name': 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
 'unit': 'kilowatt hour',
 'location': 'DK',
 'enb_location': ['EU', 'DK'],
 'default_output': {'unit': 'kWh', 'magnitude': 1}}

In [14]:
found_activities = db.search(activity_config["name"], filter={"location": "DK"})
found_activities

Excluding 104 filtered results


['heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (megajoule, DK, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (kilowatt hour, DK, None)]

In [15]:
# the 2nd activity matches the unit (index 1). We can set its 'enb_location' and save it.

In [16]:
found_activities[1]["enb_location"] = ["EU", "DK"]
found_activities[1].save()

In [17]:
# Note that enbios, uses other brightway internal mechanisms to set all locations, which is much more performant that as described above. However it also possible to call the function in the adapter that makes these changes directly
